# Phase 4: Integrate Generative AI  
In this phase, Generative AI was integrated into the system to generate personalized, data-driven sleep recommendations. This stage focuses on implementing two prompt structures and conducting templates comparison and analysis to evaluate how each template interprets the student’s sleep habits and produces tailored insights.

## 1. Implementation
In this section, two generative AI prompt templates are implemented to enhance the system’s ability to analyze and interpret student sleep patterns. Both templates interact with the LLaMA model through the Groq API.
### 1.1 Template 1: Brief Recommendation
This template generates a short, focused recommendation based on the student’s sleep habits. It identifies the main issue and provides one or two essential suggestions. The output is intentionally concise and easy to understand, making it suitable for quick feedback.
### 1.2 Template 2: Detailed Recommendation
This template offers a comprehensive evaluation of the student’s sleep behavior. It compares the user’s inputs to dataset averages, analyzes the impact of habits such as screen time, caffeine, and physical activity, and examines weekday/weekend schedule consistency. The generated recommendation includes an overview, detailed analysis, and a multi-step action plan tailored for meaningful improvement.
### 1.3 Code of Templates 1 & 2  

In [3]:
%pip install groq


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
from groq import Groq
from dotenv import load_dotenv  

# ==================================================================
# Configuration.
# ==================================================================
DATA_PATH = r"../Dataset/student_sleep_patterns_preprocessed.csv"

# Load environment variables from .env file
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY is not set in the environment.")


def load_dataset(path: str) -> pd.DataFrame:
    """Load sleep dataset."""
    df = pd.read_csv(path)
    return df


# ==================================================================
# This function computes descriptive statistics for all major attributes in the dataset.
# ==================================================================
def compute_dataset_stats(df: pd.DataFrame) -> dict:
    stats = {
        "avg_age": df["Age"].mean(),
        "gender_distribution": df["Gender"].value_counts(normalize=True).to_dict(),
        "year_distribution": df["University_Year"].value_counts(normalize=True).to_dict(),
        "avg_sleep": df["Sleep_Duration"].mean(),
        "avg_study": df["Study_Hours"].mean(),
        "avg_screen": df["Screen_Time"].mean(),
        "avg_caffeine": df["Caffeine_Intake"].mean(),
        "avg_activity": df["Physical_Activity"].mean(),
        "sleep_quality_distribution": df["Sleep_Quality"].value_counts(normalize=True).to_dict(),
        "weekday_start_mode": df["Weekday_Sleep_Start"].mode()[0],
        "weekend_start_mode": df["Weekend_Sleep_Start"].mode()[0],
        "weekday_end_mode": df["Weekday_Sleep_End"].mode()[0],
        "weekend_end_mode": df["Weekend_Sleep_End"].mode()[0],
    }
    return stats


# ==================================================================
# This function gathers all necessary inputs from the user.
# ==================================================================
def get_user_input() -> dict:
    print("=" * 60)
    print("STUDENT SLEEP PROFILE – USER INPUT".center(60))
    print("=" * 60)

    age = int(input("Age (years): "))
    gender = input("Gender (0 = Other, 1 = Male, 2 = Female): ").strip()
    uni_year = int(input("University year (1–4): "))

    sleep_duration = float(input("Average sleep duration (hours per night): "))
    study_hours = float(input("Average study hours per day: "))
    screen_time = float(input("Average screen time per day (hours): "))

    caffeine = float(input("Caffeine intake per day (cups): "))
    activity = float(input("Physical activity per week (minutes): "))

    weekday_start = input("Weekday sleep start (Early / Medium / Late): ").strip()
    weekend_start = input("Weekend sleep start (Early / Medium / Late): ").strip()
    weekday_end = input("Weekday wake-up time (Early / Medium / Late): ").strip()
    weekend_end = input("Weekend wake-up time (Early / Medium / Late): ").strip()

    user_profile = {
        "Age": age,
        "Gender": gender,
        "University_Year": uni_year,
        "Sleep_Duration": sleep_duration,
        "Study_Hours": study_hours,
        "Screen_Time": screen_time,
        "Caffeine_Intake": caffeine,
        "Physical_Activity": activity,
        "Weekday_Sleep_Start": weekday_start,
        "Weekend_Sleep_Start": weekend_start,
        "Weekday_Sleep_End": weekday_end,
        "Weekend_Sleep_End": weekend_end,
    }

    return user_profile


# ==================================================================
# Template 1: Brief Recommendation (short, focused feedback) 
# ==================================================================
def build_brief_prompt(user: dict, stats: dict) -> str:
    prompt = f"""
You are a sleep coach for university students.

Student profile:
- Age: {user["Age"]} (dataset average age: {stats["avg_age"]:.1f})
- Gender (dataset coded as 0=Other, 1=Male, 2=Female): {user["Gender"]}
- University year: {user["University_Year"]}
- Sleep duration: {user["Sleep_Duration"]:.1f} hours/night (dataset average: {stats["avg_sleep"]:.1f})
- Study hours: {user["Study_Hours"]:.1f} hours/day (dataset average: {stats["avg_study"]:.1f})
- Screen time: {user["Screen_Time"]:.1f} hours/day (dataset average: {stats["avg_screen"]:.1f})
- Caffeine intake: {user["Caffeine_Intake"]:.1f} cups/day (dataset average: {stats["avg_caffeine"]:.1f})
- Physical activity: {user["Physical_Activity"]:.0f} minutes/week (dataset average: {stats["avg_activity"]:.0f})
- Weekday sleep: {user["Weekday_Sleep_Start"]} → {user["Weekday_Sleep_End"]}
- Weekend sleep: {user["Weekend_Sleep_Start"]} → {user["Weekend_Sleep_End"]}

Task:
Give a very brief sleep recommendation (max 3–4 sentences).
Focus only on:
- The main problem in this sleep pattern.
- 1 or 2 key, practical suggestions.

Write in clear, simple English for a university student.
"""
    return prompt


# =========================================================================
# Template 2: Detailed Recommendation (full analysis & action plan) 
# =========================================================================
def build_detailed_prompt(user: dict, stats: dict) -> str:
    prompt = f"""
You are an expert sleep coach for university students.

You are given this student profile:

Demographic and academic:
- Age: {user["Age"]} (dataset average age: {stats["avg_age"]:.1f})
- Gender (0=Other, 1=Male, 2=Female): {user["Gender"]}
- University year: {user["University_Year"]}

Habits (with dataset comparison):
- Sleep duration: {user["Sleep_Duration"]:.1f} hours/night 
  (dataset average: {stats["avg_sleep"]:.1f} hours)
- Study hours: {user["Study_Hours"]:.1f} hours/day 
  (dataset average: {stats["avg_study"]:.1f} hours)
- Screen time: {user["Screen_Time"]:.1f} hours/day 
  (dataset average: {stats["avg_screen"]:.1f} hours)
- Caffeine intake: {user["Caffeine_Intake"]:.1f} cups/day 
  (dataset average: {stats["avg_caffeine"]:.1f} cups)
- Physical activity: {user["Physical_Activity"]:.0f} minutes/week 
  (dataset average: {stats["avg_activity"]:.0f} minutes)

Sleep schedule patterns (dataset modes in brackets):
- Weekday: {user["Weekday_Sleep_Start"]} sleep → {user["Weekday_Sleep_End"]} wake 
  [most common pattern in dataset: {stats["weekday_start_mode"]} → {stats["weekday_end_mode"]}]
- Weekend: {user["Weekend_Sleep_Start"]} sleep → {user["Weekend_Sleep_End"]} wake 
  [most common pattern in dataset: {stats["weekend_start_mode"]} → {stats["weekend_end_mode"]}]

Task:
Write a detailed sleep recommendation for this student. Structure your answer as:

1. Short overview (2–3 sentences) describing the main issues in this pattern.
2. Analysis:
   - Sleep duration vs dataset average.
   - Study hours and screen time impact.
   - Caffeine and physical activity impact.
   - Weekday vs weekend schedule consistency.
3. Action plan:
   - 3–5 concrete steps to improve sleep quality (with specific numbers when possible, e.g., "reduce screen time to X hours", "aim for Y minutes of activity").
4. Final encouragement message in one sentence.

Write in clear, friendly English, suitable for a university student.
"""
    return prompt


# =========================================================================
# Send a prompt to the Groq LLaMA model and return the generated response.
# =========================================================================
def call_groq(prompt: str, model_name: str = "llama-3.1-8b-instant") -> str:
    client = Groq(api_key=GROQ_API_KEY)

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful sleep coach for university students."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
        max_tokens=900,
    )

    return response.choices[0].message.content


# =========================================================================
# Print the collected user profile so it remains visible in the output cell.
# =========================================================================
def pretty_print_profile(user_profile: dict):
    print("\n" + "=" * 60)
    print("YOUR ENTERED SLEEP PROFILE".center(60))
    print("=" * 60)
    for key, value in user_profile.items():
        print(f"{key}: {value}")


# =========================================================================
# Main Function
# =========================================================================
def main():
    # Load dataset and compute stats once
    df = load_dataset(DATA_PATH)
    stats = compute_dataset_stats(df)

    # Get user profile from input
    user_profile = get_user_input()

    # Show the filled inputs in the same output cell
    pretty_print_profile(user_profile)

    # Generate Template 1: Brief Recommendation
    brief_prompt = build_brief_prompt(user_profile, stats)
    print("\n" + "=" * 60)
    print("TEMPLATE 1: BRIEF RECOMMENDATION".center(60))
    print("=" * 60)
    brief_answer = call_groq(brief_prompt)
    print(brief_answer)

    # Generate Template 2: Detailed Recommendation
    detailed_prompt = build_detailed_prompt(user_profile, stats)
    print("\n" + "=" * 60)
    print("TEMPLATE 2: DETAILED RECOMMENDATION".center(60))
    print("=" * 60)
    detailed_answer = call_groq(detailed_prompt)
    print(detailed_answer)


if __name__ == "__main__":
    main()


             STUDENT SLEEP PROFILE – USER INPUT             

                 YOUR ENTERED SLEEP PROFILE                 
Age: 24
Gender: 2
University_Year: 4
Sleep_Duration: 5.0
Study_Hours: 4.0
Screen_Time: 7.0
Caffeine_Intake: 3.0
Physical_Activity: 140.0
Weekday_Sleep_Start: Late
Weekend_Sleep_Start: Late
Weekday_Sleep_End: Early
Weekend_Sleep_End: Late

              TEMPLATE 1: BRIEF RECOMMENDATION              
Based on your profile, I'd like to highlight that your main sleep problem is going to bed late on both weekdays and weekends, which is disrupting your sleep schedule and leading to inadequate sleep duration (only 5 hours/night).

To improve your sleep, I recommend two practical suggestions:

1. Establish a consistent sleep schedule: Try to go to bed and wake up at the same time every day, including weekends, to regulate your body's internal clock.
2. Wind down before bedtime: Avoid screens (e.g., phones, laptops, or TVs) and caffeine for at least an hour before bedtime, 

 ## 2. Template Comparison & Analysis
 ### Template 1: Brief Recommendation – Analysis

Template 1 is a short-prompt structure.
It is designed to generate fast and focused sleep advice.
It uses the full student profile:

- Age  
- Gender  
- University_Year  
- Sleep_Duration  
- Study_Hours  
- Screen_Time  
- Caffeine_Intake  
- Physical_Activity  
- Weekday_Sleep_Start  
- Weekend_Sleep_Start  
- Weekday_Sleep_End  
- Weekend_Sleep_End  

The model is instructed to return only:
- the main problem  
- and one or two key suggestions.

#### Output Demonstration

In our example, the student profile is:

- Age: 24  
- Gender: 2 (Female, according to the dataset coding)  
- University_Year: 4  
- Sleep_Duration: 5.0 hours per night  
- Study_Hours: 4.0 hours per day  
- Screen_Time: 7.0 hours per day  
- Caffeine_Intake: 3.0 cups per day  
- Physical_Activity: 140.0 minutes per week  
- Weekday_Sleep_Start: Late  
- Weekend_Sleep_Start: Late  
- Weekday_Sleep_End: Early  
- Weekend_Sleep_End: Late  

When Template 1 is applied to this profile, the model produces a short answer.
The output:

- identifies the core issue as not getting enough sleep  
  (Sleep_Duration = 5.0 hours, with a 4th-year workload).  
- highlights the high Screen_Time (7.0 hours per day) before bedtime  
  as a major negative habit.  
- suggests 1–2 concrete actions,  
  such as going to bed earlier on weekdays  
  and reducing screen exposure in the evening  
  to reach about 7–8 hours of sleep.

The recommendation is brief and easy to read.

#### Quality

The quality of Template 1’s output is high in terms of clarity.
The language is simple and friendly.
It is easy for a university student to understand.

The message goes directly to:

- what the main problem is  
- and what the student should change first.

There is no long introduction.
There is no complex technical wording.
This makes the feedback suitable for quick guidance.

#### Level of Detail

Template 1 is intentionally low-detail.  
The prompt still provides many profile features and dataset statistics to the model.  
However, the instructions restrict the response to:

- one main issue  
- one or two practical, high-impact suggestions  

Because of this, not every attribute in the profile appears in the generated text.  
For example, Physical_Activity and weekend wake-up time may not be mentioned explicitly.  
Even with this limitation, the output remains focused and easy to consume.

#### Relevance

Despite being brief, the recommendation is still strongly tied to the data.
The model uses the actual values in the profile:

- Sleep_Duration = 5.0  
- Screen_Time = 7.0  
- Caffeine_Intake = 3.0  
- Late / Early sleep schedule

These values are reflected in the explanation and the advice.
The feedback does not look generic.
It clearly reacts to this specific student profile
and provides realistic steps the student can start with immediately.

## 3. Justification
